In [1]:
import ee
import geemap
import pandas as pd
import altair as alt
import numpy as np
import folium
import matplotlib.pyplot as plt


# Initialize the Earth Engine API
ee.Initialize()


In [2]:
#define aoi
#AOI = ee.FeatureCollection("projects/connect-gaea/assets/meru_county").geometry()
BOUNDARIES_ID = 'FAO/GAUL/2015/level1'
#import
rift = ee.FeatureCollection(BOUNDARIES_ID).filter(
    'ADM0_NAME == "Kenya" && '
    'ADM1_NAME == "Rift Valley"'
)

AOI = rift.geometry()



In [3]:
start_date = ee.Date('2021-01-01')
end_date = ee.Date('2021-07-01')

In [4]:
#clipping function
def clip_img(img):
    return img.clip(AOI)

#chirps data
imgCol = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").map(clip_img)

In [5]:
# Calculate the sum of precipitation during the defined timeframe
pr_sum = imgCol.filterDate(start_date, end_date).sum().set({
    'system:time_start': start_date.millis(),
    'system:time_end': end_date.advance(-1, 'day').millis()
})


In [6]:
# Calculate the number of days in the timeframe
dif_days = end_date.difference(start_date, 'day')
print('No. of days:' ,dif_days.getInfo())

# Get the month and day of the start date
start_month = start_date.get('month').getInfo()
start_day = start_date.get('day').getInfo()
print('Month of start date:', start_month)
print('Day of start date:', start_day)

# Get a list of all years for which CHIRPS data exists
col_start_year = ee.Date(ee.List(imgCol.get('date_range')).get(0)).get('year')
col_end_year = ee.Date(ee.List(imgCol.get('date_range')).get(1)).get('year')
year_list = ee.List.sequence(col_start_year, col_end_year, 1)

print('year list:', year_list.getInfo())
print('col end:', col_end_year.getInfo())
print('col start:', col_start_year.getInfo())



No. of days: 181
Month of start date: 1
Day of start date: 1
year list: [1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
col end: 2022
col start: 1981


In [7]:
# Define a function to calculate the precipitation sum for each year
def calc_pr_sum(year):
    start = ee.Date.fromYMD(year, start_month, start_day)  # Create the start date
    end = start.advance(dif_days, 'day')  # Create the end date
    filt_size = imgCol.filterDate(start, end).size()
    filt_pr_sum = imgCol.filterDate(start, end).sum()
    # Only return the summed precipitation image if there was an image for each day within the desired timeframe
    return ee.Algorithms.If(filt_size.eq(dif_days), filt_pr_sum)

# Calculate the long term mean for the defined timeframe
hist_pr_mean = ee.ImageCollection.fromImages(year_list.map(calc_pr_sum)).mean()

# Print the result (for demonstration purposes)
print(hist_pr_mean.getInfo())

{'type': 'Image', 'bands': [{'id': 'precipitation', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [8]:
# Calculate the difference between the precipitation sum and the long term mean
pr_anomaly = pr_sum.subtract(hist_pr_mean)


In [9]:
# Visualization parameters
vis_params = {
    'min': -300,
    'max': 300,
    'palette': ['d53e4f', 'fc8d59', 'fee08b', 'ffffbf', 'e6f598', '99d594', '3288bd']
}

# Create a map
Map = geemap.Map()

# Add layers to the map
Map.addLayer(pr_anomaly, vis_params, 'Precipitation anomaly')
Map.centerObject(AOI, 5)

Map.add_colorbar(
    vis_params, 
    label="Precipitation Anomaly", 
    layer_name="Anomaly", 
    orientation="vertical",
    transparent_bg=True
)

# Display the map
Map


Map(center=[1.0888949489012338, 35.95937986968731], controls=(WidgetControl(options=['position', 'transparent_…

In [10]:
# Calculate the total monthly precipitation
col_start_date = ee.Date(ee.List(imgCol.get('date_range')).get(0))
col_end_date = ee.Date(ee.List(imgCol.get('date_range')).get(1))
dif_months = col_end_date.difference(col_start_date, 'month')
list_month = ee.List.sequence(0, dif_months, 1)
def get_monthly_sum(month):
    start = col_start_date.advance(month, 'month')
    end = start.advance(1, 'month')
    return imgCol.filterDate(start, end).sum().set({
        'system:time_start': start.millis(),
        'system:time_end': end.advance(-1, 'day').millis()
    })

monthly_pr_sum = ee.ImageCollection.fromImages(list_month.map(get_monthly_sum))

In [11]:
# Check the number of images in the Image Collection
num_images = monthly_pr_sum.size()

# Print the number of images
print("Number of images in the collection:", num_images.getInfo())


Number of images in the collection: 494


In [19]:
# Calculate the total monthly precipitation
col_start_date = ee.Date(ee.List(imgCol.get('date_range')).get(0))
col_end_date = ee.Date(ee.List(imgCol.get('date_range')).get(1))
dif_months = col_end_date.difference(col_start_date, 'month')
list_month = ee.List.sequence(0, dif_months, 1)

def get_monthly_sum(month):
    start = col_start_date.advance(month, 'month')
    end = start.advance(1, 'month')
    return imgCol.filterDate(start, end).sum().set({
        'system:time_start': start.millis(),
        'system:time_end': end.advance(-1, 'day').millis()
    })

monthly_pr_sum = ee.ImageCollection.fromImages(list_month.map(get_monthly_sum))


In [21]:
# Calculate the total monthly precipitation
col_start_date = ee.Date(ee.List(imgCol.get('date_range')).get(0))
col_end_date = ee.Date(ee.List(imgCol.get('date_range')).get(1))
dif_months = col_end_date.difference(col_start_date, 'month')
list_month = ee.List.sequence(0, dif_months, 1)

def get_monthly_sum(month):
    start = col_start_date.advance(month, 'month')
    end = start.advance(1, 'month')
    monthly_precip = imgCol.filterDate(start, end).sum().set({
        'system:time_start': start.millis(),
        'system:time_end': end.advance(-1, 'day').millis()
    })
    # Reduce to region to get the mean precipitation value over AOI
    mean_precip = monthly_precip.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=AOI,
        scale=5000
    )
    return ee.Feature(None, {
        'date': start.format('YYYY-MM'),
        'precipitation': mean_precip.get('precipitation')
    })

monthly_features = list_month.map(get_monthly_sum)
monthly_fc = ee.FeatureCollection(monthly_features)


In [22]:
# Extract data to a Pandas DataFrame
features = monthly_fc.getInfo()['features']

# Convert the features to a DataFrame
data = []
for feature in features:
    properties = feature['properties']
    data.append([properties['date'], properties['precipitation']])
df = pd.DataFrame(data, columns=['date', 'precipitation'])
df['date'] = pd.to_datetime(df['date'])


In [23]:
# Ensure the DataFrame is properly formatted
df['precipitation'] = pd.to_numeric(df['precipitation'])

# Plotting with Altair
chart = alt.Chart(df).mark_line(point=True).encode(
    x='date:T',
    y='precipitation:Q',
    tooltip=['date:T', 'precipitation:Q']
).properties(
    title='Monthly Precipitation',
    width=600,
    height=400
).interactive()

chart.display()


alt.Chart(...)